In [1]:
import os
import matplotlib.pyplot as plt
import imageio

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPool2D

# Using static random seed
np.random.seed(123)

# number of images taken from each
COUNT = 100

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


#### Load Male Images

In [2]:
# Placeholder for data
male_list = list()

# Set the number taken from the male section
MALE_COUNT = COUNT

for filename in os.listdir("male/"):
    # Only use jpg files -- other types of files ignored
    if filename.endswith(".jpg"):
        
        relative_img_filename = os.path.join("male/", filename)
        
        # Create image from filename
        image = imageio.imread(relative_img_filename)
        
        # Convert image to numpy array
        npImage = np.array(image)
        
        # Add to list
        male_list.append(npImage)
        
        MALE_COUNT -= 1
        if(MALE_COUNT <= 0):
            break

# Turn male_list into numpy array
male_list = np.array(male_list)
male_list.shape

(100, 350, 350, 3)

#### Load Female Images

In [3]:
female_list = list()

# Set the number taken from the female section
FEMALE_COUNT = COUNT

for filename in os.listdir("female/"):
    if filename.endswith(".jpg"):
        relative_img_filename = os.path.join("female/", filename)
        
        # Create image from filename
        image = imageio.imread(relative_img_filename)
        
        # Convert image to numpy array
        npImage = np.array(image)
        
        # Add to list prepending label (0: male, 1: female)
        female_list.append(npImage)
        
        FEMALE_COUNT -= 1
        if(FEMALE_COUNT <= 0):
            break

# Turn male_list into numpy array
female_list = np.array(female_list)
female_list.shape

(100, 350, 350, 3)

#### Add labels (0: male, 1: female)

In [4]:
reshaped_male = list()
for item in male_list:
    reshaped_male.append((0, item))

reshaped_female = list()
for item in male_list:
    reshaped_female.append((1, item))

### Aggregate and Shuffle

In [5]:
all_data = reshaped_male + reshaped_female

In [6]:
from random import shuffle
shuffle(all_data)

### Arrange into Training / Test sets

In [7]:
# extract the labels
data = []
labels = []

In [8]:
for item in all_data:
    # Add label to labels
    labels.append(item[0])
    
    # Add data_item to data
    data.append(item[1])

In [9]:
data = np.array(data)

In [10]:
data.shape
# This should look like:
# (5500, 350, 350, 3)
# (num_imgs, width?, height?, depth) --> Depth signifies color depth or R, G, B as distinct values

(200, 350, 350, 3)

In [11]:
labels = np.array(labels)

In [12]:
len(labels)

200

#### Normalize pixel values
Current form:
- int value (0, 255)
Target form:
- float value (0.0, 1.0)

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
# Split the data
data_train, data_test, labels_train, labels_test = train_test_split(data, labels, test_size=0.20, random_state=42)

In [15]:
data_train = data_train.astype('float32')

In [16]:
# Normalize values
data_train = data_train.astype('float')
data_test = data_test.astype('float')

data_train /= 255
data_test /= 255

## Build Model

In [17]:
model = Sequential()

#### Add layers

In [18]:
model.add(Conv2D(32, (3,3), input_shape=(350,350,3), activation='relu'))
model.add(Conv2D(32, (3,3), activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
# model.add(Dense(2, activation='sigmoid')) --> original
model.add(Dense(1, activation='sigmoid'))
# model.add(Dense(1, activation='relu')) --> doesn't converge

#### Compile Model

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

#### Train Model

In [ ]:
model.fit(data_train,
          labels_train, 
          batch_size=32,
          epochs=10,
          verbose=1)

Epoch 1/10
160/160 [==============================] - 68s - loss: 4.6686 - acc: 0.4625     
Epoch 2/10
160/160 [==============================] - 73s - loss: 1.4301 - acc: 0.4875     
Epoch 3/10
160/160 [==============================] - 73s - loss: 0.6969 - acc: 0.5312     
Epoch 4/10
128/160 [=======================>......] - ETA: 14s - loss: 0.6946 - acc: 0.5703